In [4]:
# Configuration

# days used in calculating ratios
DAYS_RATIO = [3, 5, 10, 15, 30, 60, 120, 250]

# risk free rate
RISK_FREE_RATE = 0.04

In [5]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [6]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
# print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DEST_DIR) / f'df_OHLCV_{date_str}_clean.parquet'
dest_path = Path(DEST_DIR) / f'df_perf_ratios.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-03
DEST_DIR: ..\data

source_path: ..\data\df_OHLCV_2025-04-03_clean.parquet
dest_path: ..\data\df_perf_ratios.parquet


In [7]:
import pandas as pd

# Get tickers from df_finviz.pkl file
df_finviz = pd.read_parquet('..\data\df_finviz.parquet')
tickers = df_finviz.index.to_list()

In [8]:
import pandas as pd

pd.set_option('display.width', 1000) 

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# # Display the first few rows of the DataFrame to verify
# df = df[['Adj Close']].copy()
display(df)

Open    High     Low   Close  Adj Close   Volume    Adj Open    Adj High     Adj Low
Symbol Date                                                                                              
MNDY   2025-04-03  251.00  253.18  233.80  233.63     233.63  1229809  251.000000  253.180000  233.800000
       2025-04-02  250.99  269.82  250.99  266.60     266.60   644600  250.990000  269.820000  250.990000
       2025-04-01  243.87  263.80  240.73  262.18     262.18  1361500  243.870000  263.800000  240.730000
       2025-03-31  236.51  245.32  230.66  243.16     243.16   680000  236.510000  245.320000  230.660000
       2025-03-28  255.05  255.45  244.30  247.01     247.01   618300  255.050000  255.450000  244.300000
...                   ...     ...     ...     ...        ...      ...         ...         ...         ...
TRP    2024-04-10   38.06   38.06   37.00   37.16      35.94  4151100   36.810452   36.810452   35.785253
       2024-04-09   38.11   38.37   37.85   38.34      37.08  2396300   36.857559   37.109014   36.606103
       2024-04-08   38.33   38.37   37.92   38.08      36.83  4515700   37.071794   37.110481   36.675252
       2024-04-05   38.55   38.67   38.05   38.21      36.96  2610800   37.288877   37.404952   36.805234
       2024-04-04   39.39   39.55   38.67   38.74      37.47  2105600   38.098691   38.253446   37.402295

[385536 rows x 9 columns]

In [9]:
import pandas as pd

# Load the DataFrame df_OHLCV_2025-03-14_clean.pkl
# df = pd.read_pickle(source_path)
df = pd.read_parquet(source_path, engine='pyarrow')

# Display the first few rows of the DataFrame to verify
df = df[['Adj Close']].copy()
display(df)

Adj Close
Symbol Date                 
MNDY   2025-04-03     233.63
       2025-04-02     266.60
       2025-04-01     262.18
       2025-03-31     243.16
       2025-03-28     247.01
...                      ...
TRP    2024-04-10      35.94
       2024-04-09      37.08
       2024-04-08      36.83
       2024-04-05      36.96
       2024-04-04      37.47

[385536 rows x 1 columns]

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 385536 entries, ('MNDY', Timestamp('2025-04-03 00:00:00')) to ('TRP', Timestamp('2024-04-04 00:00:00'))
Data columns (total 1 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Adj Close  385536 non-null  float64
dtypes: float64(1)
memory usage: 4.5+ MB


In [11]:
import pandas as pd

def select_tickers_data(df, tickers):
  """
  Selects data for a list of tickers from a DataFrame with a MultiIndex
  where the first level is the ticker and the second level is the date.

  Args:
    df (pd.DataFrame): The input DataFrame with a MultiIndex.
    tickers (list): A list of ticker symbols to select.

  Returns:
    pd.DataFrame: A DataFrame containing only the data for the specified tickers.
                  Rows for tickers not found will not be included.
  """
  valid_tickers = [t for t in tickers if t in df.index.get_level_values(0)]  # Filter out the tickers that do not exist

  if not valid_tickers:
    print("No valid tickers found in the DataFrame. Returning an empty DataFrame.")
    return pd.DataFrame()

  try:
    ticker_df = df.loc[valid_tickers]
    return ticker_df
  except KeyError as e:
    print(f"KeyError after filtering valid tickers: {e}") #Added to help debugging.
    return pd.DataFrame()


# Example usage:
# ticker_list = ["UBS", "AAPL", "MSFT", "GEV"]  # Include some valid and invalid tickers
selected_data = select_tickers_data(df, tickers)
print(selected_data)

                   Adj Close
Symbol Date                 
AAPL   2025-04-03     203.19
       2025-04-02     223.89
       2025-04-01     223.19
       2025-03-31     222.13
       2025-03-28     217.90
...                      ...
SKYY   2024-04-10      95.01
       2024-04-09      96.46
       2024-04-08      96.00
       2024-04-05      95.40
       2024-04-04      94.15

[385536 rows x 1 columns]


In [12]:
# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

import sys

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))

import utils

list_dfs = utils.get_latest_dfs(selected_data, DAYS_RATIO)

In [13]:
import numpy as np

all_results = {}

for _df in list_dfs:
    tickers_in_df = _df.index.get_level_values(0).unique()
    for ticker in tickers_in_df:
        # Suppress division warnings just for this calculation
        with np.errstate(divide='ignore', invalid='ignore'):
            result_df = utils.analyze_stock(_df, ticker, risk_free_rate=RISK_FREE_RATE)
        
        if result_df is not None:
            # Extract the ticker name from the result_df index
            ticker_name = result_df.index[0]
            # Convert the single-row DataFrame to a dictionary
            metrics = result_df.iloc[0].to_dict()
            
            # Update the existing ticker entry or create a new one
            if ticker_name in all_results:
                all_results[ticker_name].update(metrics)
            else:
                all_results[ticker_name] = metrics

if all_results:
    combined_df = pd.DataFrame.from_dict(all_results, orient='index')
    print("\nCombined performance metrics DataFrame:")
    print(combined_df)
else:
    print("No performance metrics were calculated.")


Combined performance metrics DataFrame:
       Sharpe 3d  Sortino 3d  Omega 3d  Sharpe 5d  Sortino 5d  Omega 5d  Sharpe 10d  Sortino 10d  Omega 10d  Sharpe 15d  Sortino 15d  Omega 15d  Sharpe 30d  Sortino 30d  Omega 30d  Sharpe 60d  Sortino 60d  Omega 60d  Sharpe 120d  Sortino 120d  Omega 120d  Sharpe 250d  Sortino 250d  Omega 250d
A      -5.933714   -7.763505  0.308372  -7.411207   -7.836362  0.246685   -7.798092    -7.672415   0.238226   -6.066405    -6.197362   0.331782   -5.604851    -5.960148   0.364472   -3.314173    -3.999861   0.564921    -2.052176     -2.645400    0.706053    -1.010469     -1.327551    0.839127
AA     -7.446205   -8.953206  0.202385  -7.171558   -7.488996  0.175661   -9.287719    -8.513236   0.125782   -5.777088    -6.125219   0.353559   -3.315207    -3.851358   0.574387   -2.171557    -2.608366   0.691561    -1.450258     -1.836588    0.781222    -0.437705     -0.602774    0.930271
AAL    -6.395325   -8.148256  0.274096  -8.200898   -8.412215  0.199727   -7.

In [14]:
import numpy as np
import pandas as pd

def find_nan_inf(df):
    """
    Finds locations of NaN, Inf, and -Inf values in a DataFrame.
    
    Args:
        df (pd.DataFrame): DataFrame to check
        
    Returns:
        dict: Dictionary with column names as keys and dicts of 
              {type: indices} as values
    """
    nan_inf_locations = {}
    for col in df.columns:
        locations = {
            'NaN': df[col].index[df[col].isnull()].tolist(),
            'Inf': df[col].index[df[col] == np.inf].tolist(),
            '-Inf': df[col].index[df[col] == -np.inf].tolist()
        }
        if any(locations.values()):
            nan_inf_locations[col] = locations
    return nan_inf_locations

def print_nan_inf_locations(locations, message):
    """Prints NaN/Inf locations with a descriptive message."""
    print(message)
    if locations:
        for col, locs in locations.items():
            print(f"Column: {col}")
            for typ, indices in locs.items():
                if indices:
                    print(f"  {typ}: {indices}")
    else:
        print("No NaN or Inf values found.")

def get_column_replacement_values(df):
    """
    Calculate replacement values for each column (max for Inf/NaN, min for -Inf).
    
    Args:
        df (pd.DataFrame): Input DataFrame
        
    Returns:
        tuple: (max_values, min_values) dictionaries with column replacement values
    """
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    max_values = {}
    min_values = {}
    
    for col in numeric_cols:
        # Get finite values only
        finite_vals = df[col][np.isfinite(df[col])]
        if len(finite_vals) > 0:
            max_values[col] = finite_vals.max()
            min_values[col] = finite_vals.min()
        else:
            # If no finite values, use default
            max_values[col] = 1e5
            min_values[col] = -1e5
    
    return max_values, min_values

# Check for NaN or Inf values
has_nan_inf = combined_df.isnull().any().any() or not np.isfinite(combined_df).all().all()
print(f'combined_df has_nan_inf: {has_nan_inf}')

if has_nan_inf:
    # Calculate replacement values for each column
    max_values, min_values = get_column_replacement_values(combined_df)
    print("Replacing NaN/Inf values with column-specific max/min values")
    
    nan_inf_locations_before = find_nan_inf(combined_df)
    print_nan_inf_locations(nan_inf_locations_before, "NaN/Inf Locations BEFORE Replacement:")
    
    # Create cleaned DataFrame
    df_to_save = combined_df.copy()
    
    # Replace values column by column
    for col in df_to_save.columns:
        if col in max_values:  # Only numeric columns will be in max_values
            # Replace NaN and +Inf with max value
            df_to_save[col] = df_to_save[col].fillna(max_values[col])
            df_to_save[col] = df_to_save[col].replace([np.inf], max_values[col])
            # Replace -Inf with min value
            df_to_save[col] = df_to_save[col].replace([-np.inf], min_values[col])
else:
    print("No NaN or Inf values found.")
    df_to_save = combined_df

# Verification
print(f'df_to_save has_nan_inf: {df_to_save.isnull().any().any() or not np.isfinite(df_to_save).all().all()}')
print_nan_inf_locations(find_nan_inf(df_to_save), "NaN/Inf Locations AFTER Replacement:")

# Output cleaned DataFrame
print(df_to_save)

combined_df has_nan_inf: True
Replacing NaN/Inf values with column-specific max/min values
NaN/Inf Locations BEFORE Replacement:
Column: Sortino 3d
  Inf: ['ACI', 'AEE', 'AJG', 'ARGX', 'ASR', 'ATO', 'BBD', 'BCH', 'BEKE', 'BG', 'BJ', 'BLV', 'BNS', 'BRO', 'BSBR', 'BUD', 'CACI', 'CASY', 'CB', 'CCEP', 'CF', 'CI', 'CM', 'CME', 'CNC', 'CNP', 'COR', 'COST', 'CWEN', 'CWEN-A', 'CWST', 'DEO', 'DFCF', 'DG', 'DTE', 'ELS', 'ENB', 'ENSG', 'ERIE', 'ESLT', 'EW', 'EXC', 'FCFS', 'FE', 'FSLR', 'FTS', 'FTSM', 'GILD', 'GMAB', 'HCA', 'HDB', 'HRB', 'IAGG', 'IDA', 'IGIB', 'INCY', 'ITUB', 'JKHY', 'JNJ', 'KDP', 'KNSL', 'LMT', 'LNT', 'LW', 'MCK', 'MELI', 'MNST', 'MOH', 'MUSA', 'NTR', 'NVS', 'PAC', 'PPL', 'QGEN', 'RCI', 'RDY', 'RELX', 'RGLD', 'RLI', 'ROL', 'RPRX', 'RSG', 'RY', 'RYAN', 'SCHI', 'SIGI', 'SKM', 'SPIB', 'SUB', 'SUI', 'TAK', 'TAP', 'TIMB', 'TJX', 'TRI', 'TSCO', 'TW', 'UL', 'USFR', 'USIG', 'VCIT', 'VRSK', 'VTR', 'WCN', 'WM', 'WPM', 'XEL', 'ZBH']
Column: Omega 3d
  NaN: ['ACI', 'AEE', 'AJG', 'ARGX', 'ASR

In [15]:
ticker_w_inf = nan_inf_locations_before['Sortino 3d']['Inf'][0]
ticker_w_inf 

'ACI'

#### Use Cell below to check the inf, -inf and NaN replacement

In [16]:
print(f"combined_df.loc['ticker_w_inf']")
display(combined_df.loc[ticker_w_inf])
print(f"\ndf_to_save.loc['ticker_w_inf']")
display(df_to_save.loc[ticker_w_inf])

combined_df.loc['ticker_w_inf']


Sharpe 3d       13.088168
Sortino 3d            inf
Omega 3d              NaN
Sharpe 5d       26.244012
Sortino 5d            inf
Omega 5d              NaN
Sharpe 10d      10.896185
Sortino 10d     26.065662
Omega 10d        5.925947
Sharpe 15d       5.495474
Sortino 15d     11.731435
Omega 15d        2.454468
Sharpe 30d       2.983324
Sortino 30d      4.935422
Omega 30d        1.633549
Sharpe 60d       2.433024
Sortino 60d      3.922922
Omega 60d        1.505316
Sharpe 120d      1.948805
Sortino 120d     3.161667
Omega 120d       1.379352
Sharpe 250d      0.445860
Sortino 250d     0.673243
Omega 250d       1.078096
Name: ACI, dtype: float64


df_to_save.loc['ticker_w_inf']


Sharpe 3d          13.088168
Sortino 3d      63435.859333
Omega 3d         5652.315516
Sharpe 5d          26.244012
Sortino 5d       3518.247093
Omega 5d          444.257469
Sharpe 10d         10.896185
Sortino 10d        26.065662
Omega 10d           5.925947
Sharpe 15d          5.495474
Sortino 15d        11.731435
Omega 15d           2.454468
Sharpe 30d          2.983324
Sortino 30d         4.935422
Omega 30d           1.633549
Sharpe 60d          2.433024
Sortino 60d         3.922922
Omega 60d           1.505316
Sharpe 120d         1.948805
Sortino 120d        3.161667
Omega 120d          1.379352
Sharpe 250d         0.445860
Sortino 250d        0.673243
Omega 250d          1.078096
Name: ACI, dtype: float64

In [17]:
# Save the DataFrame to parquet file (always do this)
df_to_save.to_parquet(dest_path)
print(f"DataFrame successfully saved to {dest_path}")



DataFrame successfully saved to ..\data\df_perf_ratios.parquet


In [18]:
import pandas as pd
import numpy as np
from tabulate import tabulate

_symbols = ['USFR','AAPL', 'MSFT', 'GOOG', 'NVDA', 'IBIT', 'GLD', 'VCIT']
# Create and display the formatted table
selected_df = df_to_save.loc[_symbols] #Use df_to_save here
print(tabulate(selected_df, headers='keys', tablefmt='grid', floatfmt='.4f'))

+------+-------------+--------------+------------+-------------+--------------+------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+--------------+---------------+-------------+---------------+----------------+--------------+---------------+----------------+--------------+
|      |   Sharpe 3d |   Sortino 3d |   Omega 3d |   Sharpe 5d |   Sortino 5d |   Omega 5d |   Sharpe 10d |   Sortino 10d |   Omega 10d |   Sharpe 15d |   Sortino 15d |   Omega 15d |   Sharpe 30d |   Sortino 30d |   Omega 30d |   Sharpe 60d |   Sortino 60d |   Omega 60d |   Sharpe 120d |   Sortino 120d |   Omega 120d |   Sharpe 250d |   Sortino 250d |   Omega 250d |
+======+=============+==============+============+=============+==============+============+==============+===============+=============+==============+===============+=============+==============+===============+=============+==============+===============+====